<a href="https://colab.research.google.com/github/educarrascov/educarrascov/blob/main/Algoritmos_gen%C3%A9ticos_con_DEAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
The are a few python modules that are built specially for GA development. DEAP is one of them, in this notebook we have an introduction to the basics of this module, and we are going to use to solve a TSP.

# Installation and Imports
Firts, we install the module, and import all necesary code.

In [1]:
!pip install deap
import array
import random
import json

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

     |████████████████████████████████| 109 kB 5.6 MB/s eta 0:00:01


# Data Import
We are going to use presolved instance of the TSP (17,24 and 120 nodes). These presolved instances will help to see how good the GA performed when compared to the optimal solution.

In [2]:
!git clone https://github.com/juan-carvajal/TSP_Data.git

Cloning into 'TSP_Data'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (5/5), 28.17 KiB | 4.70 MiB/s, done.


# DEAP Genetic algoritms library
DEAP comes with a lot of predefined operators and algorithms, these can be used to solve a variety of problems, and we can also create custom operators for different problems, but here the ones that are provided by the module will be enought.

In [5]:
with open("TSP_Data/gr120.json", "r") as tsp_data:
    tsp = json.load(tsp_data)

distance_map = tsp["DistanceMatrix"]

IND_SIZE = tsp["TourSize"]


creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)

toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalTSP(individual):
    distance = distance_map[individual[-1]][individual[0]]
    for gene1, gene2 in zip(individual[0:-1], individual[1:]):
        distance += distance_map[gene1][gene2]
    return distance,

toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=30)
toolbox.register("evaluate", evalTSP)

def main():
    random.seed(169)

    pop = toolbox.population(n=1000)

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 200, stats=stats, 
                        halloffame=hof)
    
    return pop, stats, hof

if __name__ == "__main__":
    pop,stats,hof=main()
    print(hof)
    print(evalTSP(hof[0]))

gen	nevals	avg    	std    	min  	max  
0  	1000  	52274.3	2335.79	45169	58996
1  	794   	49359.4	2479.61	41873	58935
2  	773   	47486.6	2892.08	41830	56662
3  	785   	45310.3	3212.18	40274	58040
4  	752   	42904.5	2500.73	39313	56431
5  	743   	43119.1	3299.06	38614	55686
6  	748   	42616.6	3652.77	37615	55985
7  	748   	41468.6	3514.92	36344	55250
8  	737   	39485.9	2986.22	35744	52981
9  	762   	37285.1	1591.53	34300	44946
10 	760   	36333.3	1636.74	33353	45318
11 	747   	35633.8	1756.53	32992	44611
12 	786   	35034.3	1942.63	32389	44449
13 	743   	33828.6	1696.72	31896	42059
14 	745   	33280.5	1853.93	31305	44782
15 	779   	32585.5	1743.18	30892	42630
16 	779   	32186.7	1950.51	30446	41876
17 	759   	31634.8	1821.64	30038	40591
18 	766   	31210.7	1917.51	29488	41430
19 	774   	30708.2	1733.92	29247	40114
20 	745   	30353.6	1853.87	29074	40550
21 	757   	30117.5	1941.66	28821	39700
22 	761   	29965.9	1997.12	28616	40146
23 	736   	29561  	1814.51	28532	38919
24 	751   	29414.3	1954.9